In [11]:
import pandas as pd

df = pd.read_csv(r"EFX_dataset25.csv")
df = df[["EFX_NAME", "EFX_LEGSUBNAMEALL", "EFX_LEGDOMULTNAMEALL", "EFX_AFFLULTNAMEALL"]]

In [12]:
# 'Root' = 'EFX_LEGDOMULTNAMEALL'
# 'Child' = 'EFX_LEGDOMULTNAMEALL' 'EFX_LEGSUBNAMEALL'
# 'EFX_LEGSUBNAMEALL' --> 'EFX_LEGDOMULTNAMEALL' --> 'EFX_AFFLULTNAMEALL'

In [13]:
df

,EFX_NAME,EFX_LEGSUBNAMEALL,EFX_LEGDOMULTNAMEALL,EFX_AFFLULTNAMEALL
0,CIS BY DELOITTE,CIS,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
1,"DOBLIN, INC",DOBLIN INC,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
2,HEAT,HEAT INDUSTRIES,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
3,CIS BY DELOITTE,CIS,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
4,HEAT,HEAT INDUSTRIES,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
...,...,...,...,...
817,DELOITTE DIGITAL,DELOITTE CONSULTING L.P. OF WASHINGTON,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
818,"LOGIC METHOD IT, LLC",DELOITTE CONSULTING L.P. OF WASHINGTON,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
819,HEAT INDUSTRIES,DELOITTE CONSULTING L.P. OF WASHINGTON,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
820,DELOITTE DIGITAL,DELOITTE CONSULTING L.P. OF WASHINGTON,DELOITTE US DENVER,DELOITTE TOUCHE TOHMATSU LIMITED
